<a href="https://colab.research.google.com/github/ByronBlaze/nmt_jpn_eng/blob/master/NMT_ENG_JAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import tensorflow as tf
from sklearn.model_selection import train_test_split
import re
import os
import io
import time


In [3]:
os.chdir('/content/drive/My Drive/Projects/NMT')

In [87]:
text_path = 'eng_jpn2.txt'

In [86]:
df = pd.read_table(text_path, names=['source', 'target', 'comments'])
df_new = df[['source', 'target']]
df_new.to_csv('eng_jpn2.txt',sep='\t',header= False, index=None)

In [89]:
text_data = pd.read_csv(text_path,sep = '\t',names=['source', 'target'])
#text_data = pd.read_csv(text_path, header=None, names=['source', 'target'])
text_data.sample(5)

,source,target
54403,Are you seriously thinking about buying a comp...,本気であの店からコンピューターを買おうと思っているんですか？
23247,How much did that beer cost?,そのビールいくらだった？
50340,Don't go near the water until you learn how to...,泳ぎ方を覚えるまで、水に近寄ってはいけません。
36963,I haven't seen her since last month.,私は先月から彼女に会っていない。
14850,The boy started crying.,その男の子は泣き始めました。


In [37]:
text_data.drop(columns=['comments'], inplace=True)
text_data.sample(5)

,source,target
40176,Tom isn't what he was three years ago.,トムはもう３年前のトムじゃないんだ。
39369,He was involved in a traffic accident.,彼は交通事故に遭った。
32554,Is the coffee too strong for you?,コーヒーって、濃すぎた？
49944,"Beginning next week, we'll be using a new text...",来週から新しい教科書を使います。
36083,The children are playing with toys.,子供たちがおもちゃで遊んでいる。


In [41]:
np.savetxt(r'jpn_eng.txt', text_data.values, fmt='%s')

In [90]:
def clean_text(line):

  num_digits = str.maketrans('','', digits)
  line = line.lower()
  line = re.sub(" +", " ", line)
  line = re.sub("'", '', line)
  line = line.translate(num_digits)
  line = re.sub(r"([?.!,¿])", r" \1 ", line)
  line = line.rstrip().strip()
  line = 'START_ ' + line + ' _END'

  return line 


In [73]:
print(clean_text('text cleaning checker !!'))

START_ text cleaning checker  !  ! _END


In [91]:
def create_lang_pair(path,sample_size):
  
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[clean_text(l) for l in line.split('\t')] for line in lines[:sample_size]]

  return zip(*word_pairs)


In [92]:
sample_size = 60000
source, target = create_lang_pair(text_path, sample_size)

In [93]:
print(source[-1])
print(target[-1])
type(target)

START_ if someone who doesnt know your background says that you sound like a native speaker ,  it means they probably noticed something about your speaking that made them realize you werent a native speaker .  in other words ,  you dont really sound like a native speaker . _END
START_ 生い立ちを知らない人にネイティブみたいに聞こえるよって言われたら、それはおそらく、あなたの喋り方のどこかが、ネイティブじゃないと感じさせたってことだよ。つまりね、ネイティブのようには聞こえないということなんだよ。 _END


tuple

In [94]:
source_sentence_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
source_sentence_tokenizer.fit_on_texts(source)

In [95]:
source_tensor = source_sentence_tokenizer.texts_to_sequences(source)


In [97]:
source_tensor = tf.keras.preprocessing.sequence.pad_sequences(source_tensor, padding='post')

In [98]:
target_sentence_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
target_sentence_tokenizer.fit_on_texts(target)
target_tensor = source_sentence_tokenizer.texts_to_sequences(target)
target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, padding='post')


In [101]:
source_train_tensor, source_test_tensor, target_train_tensor, test_target_tensor = train_test_split(source_tensor, target_tensor, test_size=0.2)

In [102]:
batch_size = 64
df = tf.data.Dataset.from_tensor_slices((source_train_tensor, target_train_tensor)).shuffle(batch_size)
df = df.batch(batch_size, drop_remainder=True)

In [106]:
source_batch, target_batch = next(iter(df))
print(source_batch.shape)
print(target_batch.shape)

(64, 50)
(64, 5)
